<a href="https://colab.research.google.com/github/hyulianton/JaringanSyarafTiruan/blob/main/jst_Implementasi_Multilayer_Perceptron_(MLP)_Menggunakan_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Proyek ini mendemonstrasikan bagaimana membangun dan melatih Jaringan Saraf Tiruan (Neural Network) dua lapisan menggunakan _framework_ **PyTorch**. PyTorch sangat efisien karena menggunakan **Automatic Differentiation (Autograd)** untuk menangani proses Backpropagation yang rumit secara otomatis, sehingga kita bisa fokus pada arsitektur dan pelatihan.

## **Bagian 0: Setup dan Mendapatkan Data**

Kita akan menggunakan `sklearn` untuk memuat dan memproses data, dan `torch` untuk mendefinisikan model dan melatihnya.

### **Sel 0.1: Impor Library**


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

### **Sel 0.2: Mendownload, Memuat, dan Preprocessing Data**

Kita memuat _dataset_ Iris, melakukan _scaling_, dan mengubahnya menjadi _tensor_ PyTorch.


In [2]:
# 1. Muat Dataset Iris (Download dilakukan oleh load_iris)
iris = load_iris()
X = iris.data   # Fitur (4 kolom)
y = iris.target # Label (0, 1, 2)

# 2. Scaling Fitur
# Penting agar gradien descent bekerja dengan baik
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3. Membagi Data
X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# 4. Konversi ke PyTorch Tensor
# Fitur (X) harus float, Label (y) harus Long/Int untuk CrossEntropyLoss
X_train = torch.FloatTensor(X_train_np)
X_test = torch.FloatTensor(X_test_np)
y_train = torch.LongTensor(y_train_np)
y_test = torch.LongTensor(y_test_np)

print(f"Dimensi Data Pelatihan (Tensor): {X_train.shape}") # torch.Size([120, 4])
print(f"Dimensi Label Pelatihan (Tensor): {y_train.shape}")  # torch.Size([120])


Dimensi Data Pelatihan (Tensor): torch.Size([120, 4])
Dimensi Label Pelatihan (Tensor): torch.Size([120])


**Penjelasan:** Dalam PyTorch, label untuk _CrossEntropyLoss_ (klasifikasi multi-kelas) harus berupa bilangan bulat tunggal (LongTensor: 0, 1, 2) BUKAN _One-Hot Encoding_.

### **Sel 0.3: Data Loading (Batching)**

Kita menggunakan `DataLoader` untuk membagi data menjadi _Mini-Batch_, yang penting untuk efisiensi pelatihan.


In [3]:
# Gabungkan fitur dan label menjadi dataset
train_dataset = TensorDataset(X_train, y_train)

# Buat DataLoader untuk Mini-Batch
batch_size = 16
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

print(f"Jumlah Mini-Batch per Epoch: {len(train_loader)}")

Jumlah Mini-Batch per Epoch: 8


**Penjelasan:** `DataLoader` akan mengiterasi data dalam potongan 16 sampel, secara otomatis mengacak (_shuffle_) data di setiap _epoch_.

----------

## **Bagian 1: Mendefinisikan Arsitektur MLP**

Kita mendefinisikan model MLP menggunakan kelas `nn.Module` PyTorch.

### **Sel 1.1: Kelas MLP 2-Lapisan**


In [4]:
# Arsitektur Jaringan
input_size = 4
hidden_size = 10
output_size = 3 # 3 kelas Iris

class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleMLP, self).__init__()

        # Lapisan Tersembunyi (Linear + ReLU)
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()

        # Lapisan Output (Linear)
        # Catatan: Softmax diterapkan secara implisit di Loss Function
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        """Implementasi Forward Pass"""
        out = self.fc1(x) # Z1 = X @ W1 + b1
        out = self.relu(out) # A1 = ReLU(Z1)
        out = self.fc2(out) # Z2 = A1 @ W2 + b2 (Output logit)
        return out

# Inisialisasi Model
model = SimpleMLP(input_size, hidden_size, output_size)
print(model)

SimpleMLP(
  (fc1): Linear(in_features=4, out_features=10, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=10, out_features=3, bias=True)
)


**Penjelasan:**

-   `nn.Linear` secara otomatis menginisialisasi bobot (W) dan bias (b).
    
-   Metode `forward` mendefinisikan alur komputasi: X→FC1→ReLU→FC2→Output.
    

### **Sel 1.2: Definisi Loss dan Optimizer**

Kita tentukan "hakim kesalahan" (_Loss_) dan "panduan menuruni lembah" (_Optimizer_).

In [5]:
# 1. Loss Function (Fungsi Rugi)
# CrossEntropyLoss menggabungkan Softmax dan Negative Log Likelihood Loss
criterion = nn.CrossEntropyLoss()

# 2. Optimizer (Gradient Descent / Adam)
# Adam adalah versi canggih dari Gradient Descent
learning_rate = 0.01
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print(f"Loss Function: {criterion}")
print(f"Optimizer: {optimizer}")

Loss Function: CrossEntropyLoss()
Optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    weight_decay: 0
)


**Penjelasan:** `optim.Adam` akan secara otomatis menerapkan pembaruan parameter (Wbaru​=Wlama​−η⋅∂W∂L​) menggunakan gradien yang dihitung.

----------

## **Bagian 2: Pelatihan (Training Loop)**

Kita menjalankan siklus pelatihan (Forward → Loss → Backward → Update) untuk beberapa _epoch_.

### **Sel 2.1: Siklus Pelatihan**


In [6]:
epochs = 500

for epoch in range(epochs):
    for X_batch, y_batch in train_loader:

        # --- 1. Forward Pass ---
        outputs = model(X_batch)

        # --- 2. Hitung Loss ---
        loss = criterion(outputs, y_batch)

        # --- 3. Backward Pass ---
        # Mengatur semua gradien menjadi nol sebelum menghitung yang baru
        optimizer.zero_grad()
        # PyTorch menghitung dL/dW untuk semua bobot secara otomatis
        loss.backward()

        # --- 4. Update Parameter ---
        # Menerapkan Gradient Descent (menggunakan gradien yang baru dihitung)
        optimizer.step()

    if (epoch+1) % 50 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [50/500], Loss: 0.0023
Epoch [100/500], Loss: 0.0143
Epoch [150/500], Loss: 0.0488
Epoch [200/500], Loss: 0.0008
Epoch [250/500], Loss: 0.1893
Epoch [300/500], Loss: 0.0002
Epoch [350/500], Loss: 0.0205
Epoch [400/500], Loss: 0.0139
Epoch [450/500], Loss: 0.0418
Epoch [500/500], Loss: 0.0000


**Penjelasan:** Perhatikan tiga baris ajaib PyTorch:

1.  `optimizer.zero_grad()`: Harus dipanggil di awal setiap iterasi.
    
2.  `loss.backward()`: **Backpropagation otomatis!** PyTorch menghitung semua ∂W∂L​.
    
3.  `optimizer.step()`: Menerapkan pembaruan Wbaru​=Wlama​−… menggunakan gradien yang baru dihitung.
    

----------

## **Bagian 3: Evaluasi Model**

Kita mengukur kinerja model pada data uji.

### **Sel 3.1: Menghitung Akurasi**

In [7]:
# Matikan mode training (tidak perlu menghitung gradien)
model.eval()

with torch.no_grad():
    # 1. Forward Pass pada data uji
    outputs = model(X_test)

    # 2. Ambil kelas prediksi (indeks dengan skor tertinggi)
    # torch.max(outputs, 1) mengembalikan nilai max dan indeksnya
    _, predicted_labels = torch.max(outputs.data, 1)

    # 3. Hitung Akurasi
    total = y_test.size(0)
    correct = (predicted_labels == y_test).sum().item()
    accuracy = correct / total

print(f"\n--- HASIL AKHIR ---")
print(f"Total Sampel Uji: {total}")
print(f"Prediksi Benar: {correct}")
print(f"Akurasi Model pada Data Uji: {accuracy*100:.2f}%")


--- HASIL AKHIR ---
Total Sampel Uji: 30
Prediksi Benar: 30
Akurasi Model pada Data Uji: 100.00%


**Kesimpulan:** Anda telah berhasil melatih MLP di PyTorch! Proses pelatihan jauh lebih ringkas dan fokus pada **arsitektur** dan **aliran data**, karena PyTorch menangani kalkulus **Backpropagation** di balik layar, memungkinkan efisiensi tinggi—khususnya ketika berhadapan dengan **Big Data** dan jaringan yang sangat dalam.
